Import libraries and  our Data Set.

In [ ]:
import keras
import numpy as np
import pandas as pd
import random
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from sklearn.metrics import confusion_matrix
from keras.activations import relu,softmax

#loading data
(train_X,train_Y), (test_X,test_Y) = fashion_mnist.load_data()


Data Preprocessing 


In [ ]:
# reshape dataset to have a single channel with 28x28 pixel size [for using conv2D]
# -1 for total length of the data
# 28x28 is size of the Image and 1 is channel.
train_X = train_X.reshape(-1, 28,28, 1)
test_X = test_X.reshape(-1, 28,28, 1)

train_X.shape

#convert from integers to Floats
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')

#Normalize to 0-1  [as color combination 0-255]
train_X = train_X / 255.0
test_X = test_X / 255.0

# model cannot work with categorical data Directly (0-9 digits)
# Use Hot encoding to transfer this data into 0 & 1 using 10 digits (9 zeros,1 one)
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)


Making Model using Keras Libraries

In [ ]:
#Sequential groups a linear stack of layers
model = Sequential()

#to drop the unecessary data from the image matrix 
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))

# to pick the maximum element from the part of image pixel
model.add(MaxPooling2D((2, 2)))

#Flattern is used to convert it into 1D array for next layer Processing
model.add(Flatten())

# hidden fully connected layer (i.e.Dense) with 100 nodes and relu activation function
model.add(Dense(100, activation='relu'))

# to drop out data to remove overfitting
model.add(Dropout(0.3))

#final output fully connected layer (i.e. Dense) with 10 nides and Softmax activation function
model.add(Dense(10, activation='softmax'))

# compile model by adding to optimizer  with adam opt and loss function
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#fitting the data into model with 5 epochs 
model.fit(x= train_X , y = train_Y , epochs= 5)

Epoch 1/5
1875/1875 [==============================] - 33s 18ms/step - loss: 0.4723 - accuracy: 0.8328
Epoch 2/5
1875/1875 [==============================] - 33s 18ms/step - loss: 0.3178 - accuracy: 0.8871
Epoch 3/5
1875/1875 [==============================] - 33s 18ms/step - loss: 0.2767 - accuracy: 0.8997
Epoch 4/5
1875/1875 [==============================] - 33s 18ms/step - loss: 0.2503 - accuracy: 0.9089
Epoch 5/5
1875/1875 [==============================] - 33s 18ms/step - loss: 0.2314 - accuracy: 0.9155


Testing on few data


In [ ]:
#slicing few data from 
test_xx = test_X[:30]
test_yy = test_Y[:30]

#prediction of classes on sample data by our model
predicted=model.predict_classes(test_xx,verbose=0)
print("predictoin and actual values for checking, is the model correct ?")
print("predicted:\t", predicted) #predicted values
print("Actual :\t",test_yy)    #Actual VAlues

predictoin and actual values for checking, is the model correct ?
predicted:	 [9 2 1 1 6 1 4 6 5 7 4 5 8 3 4 1 2 4 8 0 2 5 7 5 1 6 6 0 9 3]
Actual :	 [9 2 1 1 6 1 4 6 5 7 4 5 7 3 4 1 2 4 8 0 2 5 7 9 1 4 6 0 9 3]


computing TN, TP, FN, FP for confusion calculation 

Calculating Precision, Recall, F1 score, Sensitivity, Specificity and Accuracy.


In [ ]:
# predict probabilities for test set
yhat_probs = model.predict(test_X, verbose=0)

# predict crisp classes for test set
yhat_classes = model.predict_classes(test_X, verbose=0)

#forming confusion matrix formation for True (test_Y) and Predicted
conf_matrix=confusion_matrix(test_Y, yhat_classes)

#count for actual value true and prediction is true
TP = np.diag(conf_matrix)


#count for actual value false but prediction is true
FP = conf_matrix.sum(axis=0) - np.diag(conf_matrix)

#count for actual value true but prediction is false
FN = conf_matrix.sum(axis=1) - np.diag(conf_matrix)

#count for actual value false and prediction is false
TN = conf_matrix.sum() - (FP + FN + TP)
  
print("TP:",TP)
print("TN:",TN)
print("FP:",FP)
print("FN:",FN)

# Precision or positive predictive value
precision = TP/(TP+FP)

# Sensitivity, recall, hit rate or true positive rate
recall = sensitivity= TP/(TP+FN)

# Specificity or true negative rate
specificity = TN/(TN+FP) 

#f1_score 
F1_scores = 2 * ( (precision * recall) / (precision + recall) )

# Overall accuracy
acc= (TP+TN)/(TP+FP+FN+TN)

TP: [835 984 852 951 862 981 703 971 989 960]
TN: [8895 8990 8848 8858 8832 8983 8786 8957 8969 8970]
FP: [105  10 152 142 168  17 214  43  31  30]
FN: [165  16 148  49 138  19 297  29  11  40]


In [ ]:
# to print in Matrix format making list of columns 
columns =[precision,specificity,F1_scores,acc,recall]

df=pd.DataFrame()

i=0 #adding column into data frame to print
for col in columns:
  df[i]=col
  i+=1

#printing dataFrame
df.columns=['precision','specificity','F1_scores','accuracy','recall=sensitivity']
df 

,precision,specificity,F1_scores,accuracy,recall=sensitivity
0,0.888298,0.988333,0.860825,0.9730,0.835
1,0.989940,0.998889,0.986961,0.9974,0.984
2,0.848606,0.983111,0.850299,0.9700,0.852
3,0.870082,0.984222,0.908743,0.9809,0.951
4,0.836893,0.981333,0.849261,0.9694,0.862
5,0.982966,0.998111,0.981982,0.9964,0.981
6,0.766630,0.976222,0.733438,0.9489,0.703
7,0.957594,0.995222,0.964250,0.9928,0.971
8,0.969608,0.996556,0.979208,0.9958,0.989
9,0.969697,0.996667,0.964824,0.9930,0.960
